In [96]:
!rm -f code_for_hw02.py*
!wget --no-check-certificate --quiet https://introml_oll.odl.mit.edu/6.036/static/homework/hw02/code_for_hw02.py
from code_for_hw02 import *
import numpy as np
import math

### Perceptron algorithm

Implement [the perceptron algorithm](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week2/perceptron/2), where

* `data` is a numpy array ($d$ by $n$)
* `labels` is numpy array ($1$ by $n$)
* `params` is a dictionary. Availiable paramaters: iterations $T$
* `hook` unused.

It should return a tuple of $\theta$ (a $d$ by 1 array) and $\theta_0$ (a 1 by 1 array).

In [39]:
def perceptron(data, labels, params = {}, hook = None):
    T = params.get('T', 100)
    d = len(data)
    n = len(data[0])
    th = np.array([[0]] * d)
    th0 = np.array([[0]])
    for iter in range(T):
        for i in range(n):
            x = np.array([data[:,i]])
            y = labels[:,i]
            if np.sign(np.dot(x, th) + th0) != y:
                th0 = th0 + y
                th = th + y * x.T
                if hook != None:
                    hook((th0, th))
        
        # axes = tidy_plot(-50, 50, -50, 50)
        # print(th)
        # print(th0)
        # plot_data(data, labels, axes)
        # plot_separator(axes, th, th0)
    return (th, th0)
                

In [40]:
test_perceptron(perceptron)

-----------Test Perceptron 0-----------
Passed! 

-----------Test Perceptron 1-----------
Passed! 



In [44]:
def averaged_perceptron(data, labels, params = {}, hook = None):
    T = params.get('T', 100)
    d = len(data)
    n = len(data[0])
    th = ths = np.array([[0]] * d)
    th0 = th0s = np.array([[0]])
    
    for iter in range(T):
        for i in range(n):
            x = np.array([data[:,i]])
            y = labels[:,i]
            if np.sign(np.dot(x, th) + th0) != y:
                th0 = th0 + y
                th = th + y * x.T
                if hook != None:
                    hook((th0, th))
            ths = ths + th
            th0s = th0s + th0
    return (ths / n / T, th0s / n / T)


In [45]:
test_perceptron(averaged_perceptron) # should be wrong but close to the answer

-----------Test Perceptron 0-----------
Test Failed.
Your code output  th: [[-9.0525], [17.5825]], th0: [[0.0]]
Expected  th: [[-9.0], [18.0]], th0: [[2.0]]


-----------Test Perceptron 1-----------
Test Failed.
Your code output  th: [[1.47], [-1.7275]], th0: [[0.0]]
Expected  th: [[0.0], [-3.0]], th0: [[0.0]]




In [46]:
def eval_classifier(learner, data_train, labels_train, data_test, labels_test):
    th, th0 = learner(data_train, labels_train)

    n = len(data_test[0])
    cnt = 0
    for i in range(n):
        x = np.array([data_test[:,i]])
        y = labels_test[:,i]

        if np.sign(np.dot(x, th) + th0) == y:
            cnt = cnt + 1
    return cnt / n

In [47]:
test_eval_classifier(eval_classifier, perceptron)

-----------Test Eval Classifier 0-----------
Passed! 

-----------Test Eval Classifier 1-----------
Passed! 



In [50]:
def eval_learning_alg(learner, data_gen, n_train, n_test, it):
    perf = 0
    for iter in range(it):
        data_train, labels_train = data_gen(n_train)
        data_test, labels_test = data_gen(n_test)
        perf = perf + eval_classifier(learner, data_train, labels_train, data_test, labels_test)
    perf /= it
    return perf

In [51]:
test_eval_learning_alg(eval_learning_alg, perceptron)

-----------Test Eval Learning Algo-----------
Passed! 



In [70]:
def xval_learning_alg(learner, data, labels, k):
    d = len(data)
    data_chunk = np.array_split(data, k, axis=1)
    lables_chunk = np.array_split(labels, k, axis=1)
    perf = 0
    for i in range(k):
        data_train = np.array([[]] * d)
        lables_train = np.array([[]])
        for j in range(k):
            if i != j:
                data_train = np.concatenate((data_train, data_chunk[j]), axis=1)
                lables_train = np.concatenate((lables_train, lables_chunk[j]), axis=1)
        perf = perf + eval_classifier(learner, data_train, lables_train, data_chunk[i], lables_chunk[i])
    return perf / k

In [72]:
test_xval_learning_alg(xval_learning_alg, perceptron)

-----------Test Cross-eval Learning Algo-----------
Test Failed.
Your code output  0.71
Expected  0.61




In [85]:
display(eval_learning_alg(perceptron,           gen_flipped_lin_separable(pflip=0.1), 20, 100, 100))
display(eval_learning_alg(averaged_perceptron,  gen_flipped_lin_separable(pflip=0.1), 20, 100, 100))
display(eval_learning_alg(perceptron,           gen_flipped_lin_separable(pflip=0.25), 20, 100, 100))
display(eval_learning_alg(averaged_perceptron,  gen_flipped_lin_separable(pflip=0.25), 20, 100, 100))

0.7754999999999999

0.8188999999999999

0.5924000000000001

0.6522999999999998

In [82]:
def eval_training_acc(learner, data_gen, n_train, it):
    perf = 0
    for iter in range(it):
        data_train, labels_train = data_gen(n_train)
        perf = perf + eval_classifier(learner, data_train, labels_train, data_train, labels_train)
    perf /= it
    return perf

In [87]:
display(eval_training_acc(perceptron,           gen_flipped_lin_separable(pflip=0.1), 20, 100))
display(eval_training_acc(averaged_perceptron,  gen_flipped_lin_separable(pflip=0.1), 20, 100))
display(eval_training_acc(perceptron,           gen_flipped_lin_separable(pflip=0.25), 20, 100))
display(eval_training_acc(averaged_perceptron,  gen_flipped_lin_separable(pflip=0.25), 20, 100))

0.8285000000000001

0.8754999999999998

0.6725000000000001

0.7345000000000002

In [124]:
def decay_perceptron(data, labels, params = {}, hook = None):
    T = params.get('T', 100)
    d = len(data)
    n = len(data[0])
    th = np.array([[0]] * d)
    th0 = np.array([[0]])

    def decay_func(t):
        return 0.5 - 0.4 * math.exp(-t)

    for iter in range(T):
        w = decay_func((iter + 1) / T)
        for i in range(n):
            x = np.array([data[:,i]])
            y = labels[:,i]
            if np.sign(np.dot(x, th) + th0) != y:
                th0 = th0 + y * w
                th = th + y * x.T * w
                if hook != None:
                    hook((th0, th))
    return (th, th0)
                

In [133]:
def weighted_average_perceptron(data, labels, params = {}, hook = None):
    T = params.get('T', 100)
    d = len(data)
    n = len(data[0])
    th = ths = np.array([[0]] * d)
    th0 = th0s = np.array([[0]])
    sumw = 0
    def reverse_decay_func(t):
        return 0.5 + 0.4 * math.exp(-t)
        
    for iter in range(T):
        for i in range(n):
            x = np.array([data[:,i]])
            y = labels[:,i]
            if np.sign(np.dot(x, th) + th0) != y:
                th0 = th0 + y
                th = th + y * x.T
                if hook != None:
                    hook((th0, th))
            
            t = (iter * n + i + 1) / T
            w = reverse_decay_func(t)
            ths = ths + th * w
            th0s = th0s + th0 * w
            sumw += w
    return (ths / sumw, th0s / sumw)

In [137]:
print("learning accuracy: flip = 0")
display(eval_learning_alg(perceptron,                   gen_flipped_lin_separable(pflip=0), 500, 100, 20))
display(eval_learning_alg(averaged_perceptron,          gen_flipped_lin_separable(pflip=0), 500, 100, 20))
display(eval_learning_alg(decay_perceptron,             gen_flipped_lin_separable(pflip=0), 500, 100, 20))
display(eval_learning_alg(weighted_average_perceptron,  gen_flipped_lin_separable(pflip=0), 500, 100, 20))
print("learning accuracy: flip = 0.1")
display(eval_learning_alg(perceptron,                   gen_flipped_lin_separable(pflip=0.1), 500, 100, 20))
display(eval_learning_alg(averaged_perceptron,          gen_flipped_lin_separable(pflip=0.1), 500, 100, 20))
display(eval_learning_alg(decay_perceptron,             gen_flipped_lin_separable(pflip=0.1), 500, 100, 20))
display(eval_learning_alg(weighted_average_perceptron,  gen_flipped_lin_separable(pflip=0.1), 500, 100, 20))
print("learning accuracy: flip = 0.25")
display(eval_learning_alg(perceptron,                   gen_flipped_lin_separable(pflip=0.25), 500, 100, 20))
display(eval_learning_alg(averaged_perceptron,          gen_flipped_lin_separable(pflip=0.25), 500, 100, 20))
display(eval_learning_alg(decay_perceptron,             gen_flipped_lin_separable(pflip=0.25), 500, 100, 20))
display(eval_learning_alg(weighted_average_perceptron,  gen_flipped_lin_separable(pflip=0.25), 500, 100, 20))
print("learning accuracy: flip = 0.5")
display(eval_learning_alg(perceptron,                   gen_flipped_lin_separable(pflip=0.5), 500, 100, 20))
display(eval_learning_alg(averaged_perceptron,          gen_flipped_lin_separable(pflip=0.5), 500, 100, 20))
display(eval_learning_alg(decay_perceptron,             gen_flipped_lin_separable(pflip=0.5), 500, 100, 20))
display(eval_learning_alg(weighted_average_perceptron,  gen_flipped_lin_separable(pflip=0.5), 500, 100, 20))

learning accuracy: flip = 0


0.9964999999999999

0.9984999999999999

0.9995

0.9984999999999999

...

learning accuracy: flip = 0.25


0.601

0.7095

0.5479999999999999

0.7215